In [1]:
!pip install janitor
!pip install ConfigParser
import pandas as pd
import altair as alt
import gzip
import os

  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.9/1.9 MB 8.3 MB/s eta 0:00:000m eta 0:00:01:01:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 249.3/249.3 KB 8.2 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
Using legacy 'setup.py install' for janitor, since package 'wheel' is not installed.
Using legacy 'setup.py install' for zope.interface, since package 'wheel' is not installed.
  Running setup.py install for zope.interface ... done
  Running setup.py install for janitor ... done


In [ ]:
df = pd.read_csv('/folium_test/top20ImportExportdataset.csv')

In [ ]:
df.head()

,Country,Year,Country1,Exports1,Imports1,NetExports1,Country2,Exports2,Imports2,NetExports2
0,Australia,2014,China,90132200000,49581800000,40550400000,Australia,49581800000,90132200000,-40550400000
1,Australia,2014,Japan,48127300000,16449400000,31677900000,Australia,16449400000,48127300000,-31677900000
2,Australia,2014,United States,10671800000,25641300000,-14969500000,Australia,25641300000,10671800000,14969500000
3,Australia,2014,South Korea,20413100000,11255400000,9157700000,Australia,11255400000,20413100000,-9157700000
4,Australia,2014,Germany,1770850000,11341700000,-9570850000,Australia,11341700000,1770850000,9570850000


In [ ]:
coordinates = {"Australia":[-33.8469759,150.37],
              "Brazil":[-23.6815314,-46.8],
              "Canada": [43.7184038,-79.5],
              "China": [35.86166,104.195397], 
              "France":[48.8589507,2.27],
              "Germany":[52.5069312,13.14],
              "India":[19.0825223,72.741],
              "Indonesia":[-0.789275,113.921327],
              "Iran":[32.427908,53.688046],
              "Italy":[41.87194,12.56738],
              "Japan":[36.204824,138.252924],
              "Mexico":[23.634501,-102.552784],
              "Russia":[55.5815244,36.82],
              "Netherlands":[52.132633,5.291266],
              "Saudi Arabia":[23.885942,45.079162],
              "South Korea":[35.907757,127.766922],
              "Spain":[40.463667,-3.74922],
              "Switzerland":[46.818188,8.227512],
              "United Kingdom":[51.5287352,-0.3817],
              "United States":[38.89378,-77.15]} 

In [ ]:
source = alt.topo_feature(data.world_110m.url, 'countries')

import altair as alt
from vega_datasets import data
source = alt.topo_feature(data.world_110m.url, 'countries')
alt.Chart(source).mark_geoshape(
    fill='blue',
    stroke='grey',).encode(tooltip='id:N').project('naturalEarth1').properties(width=800, height=600).configure_view(stroke=None)

alt.Chart(...)

In [ ]:
import folium
import pandas as pd

In [ ]:
world_map = folium.Map(
    width=1000,
    height=500,
    zoom_start=2,
    location=[13.133932434766733, 16.103938729508073])
world_map

In [ ]:
type(coordinates)

dict

In [ ]:
countries = pd.DataFrame(coordinates)
countries = countries.T
countries = countries.reset_index(drop=False)
countries.columns = ['name','latitude','longitude']

In [ ]:
for _, c in countries.iterrows():
    folium.Marker(
        location=[c['latitude'], c['longitude']],
        popup=c['name'],
        tooltip=c['name'],
    ).add_to(world_map)

world_map

In [ ]:
df.head()

,Country,Year,Country1,Exports1,Imports1,NetExports1,Country2,Exports2,Imports2,NetExports2
0,Australia,2014,China,90132200000,49581800000,40550400000,Australia,49581800000,90132200000,-40550400000
1,Australia,2014,Japan,48127300000,16449400000,31677900000,Australia,16449400000,48127300000,-31677900000
2,Australia,2014,United States,10671800000,25641300000,-14969500000,Australia,25641300000,10671800000,14969500000
3,Australia,2014,South Korea,20413100000,11255400000,9157700000,Australia,11255400000,20413100000,-9157700000
4,Australia,2014,Germany,1770850000,11341700000,-9570850000,Australia,11341700000,1770850000,9570850000


In [ ]:
df.loc[(df['Country'] == 'Australia') & (df['Year'] == 2020)].sort_values(['Exports1'], ascending=False).iloc[0,3]

114767000000

In [ ]:
trade_bal = {}
for key in df['Country'].unique():
  trade_hist = df.loc[(df['Country'] == key) & (df['Year'] == 2020)]
  top_export_country = df.loc[(df['Country'] == key) & (df['Year'] == 2020)].sort_values(['Exports1'], ascending=False).iloc[0,2]
  top_export         = df.loc[(df['Country'] == key) & (df['Year'] == 2020)].sort_values(['Exports1'], ascending=False).iloc[0,3]
  top_import_country = df.loc[(df['Country'] == key) & (df['Year'] == 2020)].sort_values(['Imports1'], ascending=False).iloc[0,2]
  top_import         = df.loc[(df['Country'] == key) & (df['Year'] == 2020)].sort_values(['Imports1'], ascending=False).iloc[0,3]
  trade_bal[key] = [2020, top_export_country, top_export, top_import_country, top_import]

In [ ]:
trade_bal_df = pd.DataFrame(trade_bal).T.reset_index()
trade_bal_df.columns = ['name','year','top_export_partner','total_export','top_import_partner','total_import']

In [ ]:
## Merge Pop up stats with coordinates
display_text = pd.merge(countries, trade_bal_df, on='name')
display_text.head()

,name,latitude,longitude,year,top_export_partner,total_export,top_import_partner,total_import
0,Australia,-33.846976,150.370000,2020,China,114767000000,China,114767000000
1,Brazil,-23.681531,-46.800000,2020,China,84021200000,China,84021200000
2,Canada,43.718404,-79.500000,2020,United States,270382000000,United States,270382000000
3,China,35.861660,104.195397,2020,United States,435449000000,Japan,163920000000
4,France,48.858951,2.270000,2020,Germany,67834400000,Germany,67834400000


In [ ]:
points = list(zip(display_text.latitude, display_text.longitude, display_text.name, display_text.year, display_text.top_export_partner,display_text.total_export))


In [ ]:
points[1]

(-23.6815314, -46.8, 'Brazil', 2020, 'China', 84021200000)

In [ ]:
def add_markers_to_the_map(map, display_text, color, icon):  
    points = list(zip(display_text.latitude, display_text.longitude, display_text.name, display_text.year, display_text.top_export_partner,display_text.total_export))
    for point in points:         
            popup_text = "In {}, {}'s top export partner is {} with ${}".format(str(point[3]), str(point[2]), str(point[4]), str(point[-1]))
        
            popup = folium.Popup(popup_text)
                
            marker = folium.Marker(location=[point[0], point[1]], 
                                   popup=popup, 
                                   icon = folium.Icon(color=color, icon=icon, prefix='fa')).add_to(world_map)
    return map

In [ ]:
add_markers_to_the_map(world_map, display_text, color = 'Black' , icon = 'bookmark')